In [5]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
import numpy as np

In [4]:
class MA():
    def __init__(self,win_N,isMid=False):
        self.win = [np.zeros(63) for i in range(win_N)]
        self.win_N = win_N
        self.isMid = isMid
    def __call__(self,x):
        self.win += [x]
        self.win.pop(0)
        if not self.isMid:
            return np.mean(np.array(self.win),0)
        else:
            return np.median(np.array(self.win),0)

In [23]:
trace = []
ma = MA(10)




cap = cv2.VideoCapture(0)
with mp_hands.Hands(model_complexity=0,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
#                 print(distance(hand_landmarks))
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
        # Flip the image horizontally for a selfie-view display.
        image = cv2.flip(image, 1)
        try:
            dist = round(distance(hand_landmarks),2)
#             dist = ma(dist)
            if dist<42:
                center_p = center_point(hand_landmarks)
                trace.append(center_p)
            else:
                trace = []
            image = draw_trace(image,trace)
#             image = cv2.putText(image,'hello world',(50, 355),
#                                 cv2.FONT_HERSHEY_SIMPLEX,
#                                 1, (0, 0, 255), 1)
#             image = cv2.putText(image,'dist'+str(round(center_p[0],2)),(50, 355), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)
#             image = cv2.putText(image,'dist'+str(round(center_p[1],2)),(50, 385), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)

#             image = cv2.putText(image,'dist'+str(dist),(50, 255), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
        except:
            pass
        image = cv2.putText(image,
                            'dist:'+str(round(distance(hand_landmarks),2)),
                            (50, 385), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            1, (0, 255, 0), 1)

        cv2.imshow('MediaPipe Hands', image)
        if cv2.waitKey(1) & 0xFF == ord('q'): # 一帧显示一毫秒，通过上面的while循环不断地显示下一帧，从而形成动态的视频；按q键退出循环，关闭视频。
            break
cap.release()
cv2.destroyAllWindows()

In [21]:
hand_landmarks.landmark[0].x

0.30809515714645386

In [22]:
def distance(hand_landmarks):
    import math
    x4 = hand_landmarks.landmark[4].x
    y4 = hand_landmarks.landmark[4].y
    x8 = hand_landmarks.landmark[8].x
    y8 = hand_landmarks.landmark[8].y
    try:
        res = math.sqrt((640*(x4-x8))**2+(480*(y4-y8))**2)
    except:
        res = 1000
    return res

In [8]:
def center_point(hand_landmarks):
    x4 = hand_landmarks.landmark[4].x
    y4 = hand_landmarks.landmark[4].y
    x8 = hand_landmarks.landmark[8].x
    y8 = hand_landmarks.landmark[8].y
    point = (1-x4+1-x8)/2*640,(y4+y8)/2*480
#     point = (1-x4)*640,y4*480
    return point

In [9]:
def draw_trace(img,trace):
    points_n = len(trace)
    for idx in range(points_n-1):
        img = cv2.line(img,
                       (int(trace[idx][0]),int(trace[idx][1])),
                       (int(trace[idx+1][0]),int(trace[idx+1][1])), (255, 0, 0), thickness=3)
    return img

In [38]:
image.shape

(480, 640, 3)